# Comparing the performance of various methods in assembling lists, generators and iterators for a simple case of iterables of 1 to 1e6

In [19]:
# %gvim

<IPython.core.display.Javascript object>

Cell contents can now be edited via Gvim. From command mode use 'g' to open current cell contents in Gvim. After ':wq' from Gvim, use 'u' in command mode to update cell contents.


In [10]:
from functools import reduce
import operator
import statistics as st
from scipy.stats import t

# supporting (t - test) functions

In [7]:
def sidak(alpha, c):
    """Return alpha with Sidak correction, for s-1 successive t tests.
    Args:
        alpha (float): type 1 error rate between single test pair
        c (int): number of successive t-tests to be performed
    """
    return 1 - (1 - alpha)**(1/c)

def multiple_t(options, alpha_sidak):
    """Perform s-1 t-tests between mean execution time point estimates of 1) most performant point estimate and 2) all others
    Args:
        options (dict): {"for loop scenario":(mean, sd, df),...}
        alpha_sidak: significance wrt. each test pair. (sidak correction applied)
    Return: 
        a list of option names (keys), of the significantly most performant.
    """
    disregarded = []  # significantly slower
    
    pi1 = reduce(lambda i,j: i if i[1][0] < j[1][0] else j ,options.items())[0] # get the most performant option
    
    for pi2 in options.keys():
        if pi1 == pi2:
            continue
        if is_significant((pi1, *options[pi1]), (pi2, *options[pi2]), alpha_sidak):
            disregarded.append(pi2)  # pi1 significantly bigger (slower)

    # return options minus significantly worse
    return list(set(options.keys()) - set(disregarded))

def is_significant(pi1, pi2, alpha_sidak):
    """Return True if the (pi2 - pi1) - E > 0 (i.e. pi2 is significantly longer).
    Args:
       pi1(list): (name, mean, sd, df) of option 1
       pi2(list): (name, mean, sd, df) of option 2
    """
    name1, mean1, sd1, df1 = pi1
    name2, mean2, sd2, df2 = pi2

    sd = st.sqrt(sd1**2 / (df1+1) + sd2**2 / (df2+1))
    df = min(df1, df2)

    # is pi1 significantly larger(slower))?
    CI_lower = mean2 - mean1 - t.ppf(1 - alpha_sidak, df) * sd
    
    if  CI_lower > 0:
        print(f"CI: {CI_lower} < {name1}_mean - {name2}_mean < +INF")
        return True
    else:
        return False

# sum vs ... 

In [2]:
%%timeit -n 1 -r 100
a = sum(range(1000000))

The slowest run took 4.39 times longer than the fastest. This could mean that an intermediate result is being cached.
20.5 ms ± 8.91 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [8]:
%%timeit -n 1 -r 100
b = reduce(operator.add, range(1000000))

The slowest run took 5.21 times longer than the fastest. This could mean that an intermediate result is being cached.
55.5 ms ± 21.3 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


## which are the significantly most faster?

In [11]:
options = {"sum": (20.5, 8.91, 100), "reduce":(55.5, 21.3, 100)}
alpha_sidak = sidak(0.05, 1)
print(f"\nsignificantly fastest options: {multiple_t(options, alpha_sidak)}")

CI: 31.18579419732333 < sum_mean - reduce_mean < +INF

significantly fastest options: ['sum']


Thus, sum() is *significantly* faster than reduce().

# Creating Lists ....(as opposed to iterator/ generator objects) 

### for loop 

In [12]:
%%timeit -n 1 -r 100
results = []
for i in range(1000000):
    results.append(2*i)

111 ms ± 27.5 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### while loop

In [13]:
%%timeit -n 1 -r 100
results = []
i = 0
while i < 1000000:
    results.append(2*i)
    i += 1

168 ms ± 16.8 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### list(map)

In [14]:
%%timeit -n 1 -r 100
results = list(map(lambda i: 2*i, range(1000000)))

117 ms ± 11.6 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### list(generator exp)

In [15]:
%%timeit -n 1 -r 100
results = list((i for i in range(1000000)))

61.4 ms ± 3.11 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### list comprehension

In [16]:
%%timeit -n 1 -r 100
results = [i for i in range(1000000)]

44.6 ms ± 2.58 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### significance via multiple t-tests

In [17]:
options = {"for":(111, 27.5, 100), "while": (168, 16.8, 100), "list(map())": (117, 11.6, 100), "list(generator exp)":(61.4, 3.11, 100), "list comp": (44.6, 2.58, 100)}
alpha_sidak = sidak(0.05, 4)
print(f"\nsignificantly fastest options: {multiple_t(options, alpha_sidak)}")

CI: 60.166840878827415 < list comp_mean - for_mean < +INF
CI: 119.56430868038903 < list comp_mean - while_mean < +INF
CI: 69.7182693686234 < list comp_mean - list(map())_mean < +INF
CI: 15.888103119641642 < list comp_mean - list(generator exp)_mean < +INF

significantly fastest options: ['list comp']


## Filtering

In [18]:
%%timeit -n 1 -r 100
squares = [i for i in range(1000000) if i%2==0 ]

The slowest run took 4.81 times longer than the fastest. This could mean that an intermediate result is being cached.
77.9 ms ± 26.8 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [19]:
%%timeit -n 1 -r 100
list(filter(lambda i: i%2==0, range(1000000)))

125 ms ± 10.8 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [20]:
%%timeit -n 1 -r 100
squares = list((i for i in range(1000000) if i%2==0))

92.2 ms ± 9.99 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### significance via t-tests

In [21]:
options = {"list comp":(77.9, 26.8, 100), "list(filter)": (125, 10.8, 100), "list(generator)":(92.2, 9.99, 100)}
alpha_sidak = sidak(0.05, 2)
print(f"\nsignificantly fastest options: {multiple_t(options, alpha_sidak)}")

CI: 41.4120892050519 < list comp_mean - list(filter)_mean < +INF
CI: 8.669745265285776 < list comp_mean - list(generator)_mean < +INF

significantly fastest options: ['list comp']


# Creating iterables

## generator vs map vs list comp, with subsequent iterable used by list-comp

### list comprehension

In [22]:
%%timeit -n 1 -r 100
result = [i for i in range(1000000)]
result2 = [2*i for i in result]

132 ms ± 24.6 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### map (lambda)

In [23]:
%%timeit -n 1 -r 100
result = map(lambda i: i, range(1000000))
result = [2*i for i in result]

147 ms ± 19.2 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### map (fun)

In [24]:
%%timeit -n 1 -r 100

def f(i):
    return i

result = map(f, range(1000000))
result = [2*i for i in result]

138 ms ± 14.9 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### generator (fun)

In [30]:
%%timeit -n 1 -r 100
def generator():
    for i in range(1000000):
        yield i
result2 = [2*i for i in generator()]

110 ms ± 25 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


### generator (expression)

In [29]:
%%timeit -n 1 -r 100
result = (i for i in range(1000000))
result2 = [2*i for i in result]

110 ms ± 18.1 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [31]:
options = {"list comp":(132, 24.6, 100), "map (lambda)": (147, 19.2, 100), "map (fun)": (138, 14.9, 100), "generator (fun)":(110, 25, 100), "generator (exp)": (110, 25, 100)}
alpha_sidak = sidak(0.05, 4)
print(f"\nsignificantly fastest options: {multiple_t(options, alpha_sidak)}")

CI: 14.084955054179066 < generator (exp)_mean - list comp_mean < +INF
CI: 29.886438054125172 < generator (exp)_mean - map (lambda)_mean < +INF
CI: 21.432246363658663 < generator (exp)_mean - map (fun)_mean < +INF

significantly fastest options: ['generator (fun)', 'generator (exp)']


# Reducing operations

In [32]:
%%timeit -n 1 -r 100
result = reduce(lambda i,j : i if i >j else j, range(1000000))

96.7 ms ± 14.6 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [33]:
%%timeit -n 1 -r 100
result = 0
for i in range(1000000):
    if i > result: result = i

49.9 ms ± 4.42 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [34]:
options = {"reduce()":(96.7, 14.6, 100), "for loop": (49.9, 4.42, 100)}
alpha_sidak = sidak(0.05, 1)
print(f"\nsignificantly fastest options: {multiple_t(options, alpha_sidak)}")

CI: 44.27998263069979 < for loop_mean - reduce()_mean < +INF

significantly fastest options: ['for loop']


# Filtering operations

_then using the resulting iterable in list comprehenson_

In [35]:
%%timeit -n 1 -r 100
result = filter(lambda i: i > 500000, range(1000000))
result2 = [i**2 for i in result]

230 ms ± 34.1 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [36]:
%%timeit -n 1 -r 100
result = []
for i in range(1000000):
    if i > 500000: result.append(i)
        
result2 = [i**2 for i in result]

194 ms ± 5.69 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [37]:
%%timeit -n 1 -r 100
result = [i for i in range(1000000) if i > 500000]
result2 = [i**2 for i in result]

182 ms ± 5.52 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [38]:
%%timeit -n 1 -r 100
result = (i for i in range(1000000) if i > 500000)
result2 = [i**2 for i in result]

181 ms ± 5.67 ms per loop (mean ± std. dev. of 100 runs, 1 loop each)


In [39]:
options = {"filter()":(230, 34.1, 100), "for loop": (194, 5.69, 100), "list comp":(182, 5.52, 100), "gen exp":(182, 5.67, 100)}
alpha_sidak = sidak(0.05, 3)
print(f"\nsignificantly fastest options: {multiple_t(options, alpha_sidak)}")

CI: 40.60211671885891 < gen exp_mean - filter()_mean < +INF
CI: 10.280924315284492 < gen exp_mean - for loop_mean < +INF

significantly fastest options: ['gen exp', 'list comp']
